# Lab 04 - Bayesian Fitting
## Tasks
- Construct a Gaussian Process model and tune hyperparameters of GP model given noisy data
- Investigate what kernels can be used to best represent the data

# Set up environment

In [1]:
!pip install git+https://github.com/uspas/2021_optimization_and_ml --quiet

In [2]:
%reset -f

import numpy as np
import matplotlib.pyplot as plt

#matplotlib graphs will be included in your notebook, next to the code:
%matplotlib inline

import torch
import gpytorch

## Import Data
We are going to look at some data that was generated by sampling a 5 x 5 x 5 grid in the domain [0,1] on each axis. The function that generated this data is

$$
f(x_1,x_2,x_3) = \sin(2\pi x_1)\sin(\pi x_2) + x_3
$$

The columns of the imported array is $(x_1,x_2,x_3,f)$. We need to convert it to a torch tensor to use with GPyTorch.

In [4]:
x = np.linspace(0,1,5)
xx = np.meshgrid(x,x,x)
train_x = np.vstack([ele.ravel() for ele in xx]).T
train_f = np.sin(2*np.pi*train_x[:,0]) * np.sin(np.pi*train_x[:,1]) + train_x[:,2] + np.random.randn(train_x.shape[0]) * 0.01

train_x = torch.from_numpy(train_x)
train_f = torch.from_numpy(train_f)

## Define a GP Model
Here we define an Exact GP model using GPyTorch. The model is exact because we have analytic expressions for the integrals associated with the GP likelihood and output distribution. If we had a non-Gaussian likelihood or some other complication that prevented analytic integration we can also use Variational/Approximate/MCMC techniques to approximate the integrals necessary.

Taking a close look at the model below we see two important modules:
- ```self.mean_module``` which represents the mean function
- ```self.covar_module``` which represents the kernel function (or what is used to calculate the kernel matrix

Both of these objects are torch.nn.Module objects (see https://pytorch.org/docs/stable/generated/torch.nn.Module.html). PyTorch modules have trainable parameters which we can access when doing training. By grouping the modules inside another PyTorch module (gpytorch.models.ExactGP) lets us easily control which parameters are trained and which are not. 

In [5]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_f, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_f, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

Here we initialize our model with the training data and a defined likelihood (also a nn.Module) with a trainable noise parameter.

In [6]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_f, likelihood)

NOTE: All PyTorch modules (including ExactGPModel) have ```.train()``` and ```.eval()``` modes. ```train()``` mode is for optimizing model hyperameters. ```.eval()``` mode is for computing predictions through the model posterior.

## Training the model
Here we train the hyperparameters of the model (the parameters of the covar_module and the mean_module) to maximize the marginal log likelihood (minimize the negative marginal log likelihood). Note that since everything is defined in pyTorch we can use Autograd functionality to get the derivatives which will speed up optimization using the modified gradient descent algorithm ADAM.

Also note that several of these hyperparameters (lengthscale and noise) must be strictly positive. Since ADAM is an unconstrained optimizer (which optimizes over the domain (-inf, inf)) gpytorch accounts for this constraint by optimizing the log of the lengthscale (raw_lengthscale). To get the actual lengthscale just use ```model.covar_module.base_kernel.lengthscale.item()```

<div class="alert alert-block alert-info">
    
**Task:** 
    Write the steps for minimizing the negative log likelihood using pytorch. Refer back to Lab 3 for a reminder of how to do this. Use `gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)` as the loss function (which we are trying to maximize!). Use your function to train the model and report the marginal log likelihood.
    
</div>

In [7]:
def train_model(model, likelihood):
    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    

    for i in range(training_iter):
        pass


    #print the new trainable parameters
    for param in model.named_parameters():
        print(f'{param[0]} : {param[1]}')

    return loss

<div class="alert alert-block alert-info">

**Task:** 
    Define a new GP model that uses a different kernel (or combination of kernels) to maximize the marginal log likelihood.
    
</div>

In [9]:
class MyExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_f, likelihood):
        super(MyExactGPModel, self).__init__(train_x, train_f, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        
        
                                                        
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

<div class="alert alert-block alert-info">

**Task:** 
    Plot the mean and uncertainty along the $x_1$ axis where $x_2=\pi/2, x_3=0$.
    
</div>

In [11]:
#Hint: you can use the following code to get the predicted mean, lower + upper confidence bounds
x = torch.zeros(1,3).double()
my_likelihood.eval()
my_model.eval()
with torch.no_grad():
    post = my_likelihood(my_model(x))
    mean = post.mean
    lower,upper = post.confidence_region()